### Do the imports
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


In [1]:
import os
import glob
import sys

from cellpose import models
import napari
import torch

import numpy as np
import skimage as ski

import sutils

Create the napari viewer to visualize images and mask

In [2]:
viewer = napari.Viewer()

## Load data and cellpose model

Find the image files to work with. Different path for Window, Mac, and Linux

In [3]:
cells = ski.io.imread('files/cellpose/Data/120_img.png')
cells.shape

(384, 512, 3)

In [4]:
nuclei = cells[..., 0]
cyto = cells[..., 1]


Add the image to the napari viewer

In [5]:
viewer.layers.clear()

viewer.add_image(cyto, colormap='green', blending='additive')
viewer.add_image(nuclei, colormap='magenta', blending='additive')



<Image layer 'nuclei' at 0x23d93a57b20>

The nuclei image is very hazy - do a background subtration using white tophat.  

In [6]:
bs_nuc = ski.morphology.white_tophat(nuclei, ski.morphology.disk(10))

In [7]:
viewer.add_image(bs_nuc)

<Image layer 'bs_nuc' at 0x23d9e190eb0>

### Load model

In [8]:
if sys.platform == 'darwin': 
    d = torch.device('mps')
    nuclei_model = models.Cellpose(gpu=False, device=d, model_type='nuclei')
    cyto_model = models.Cellpose(gpu=False, device=d, model_type='cyto2')
else:   
    nuclei_model = models.Cellpose(gpu=True, model_type='nuclei')
    cyto_model = models.Cellpose(gpu=True, model_type='cyto2')


## Segment nucleii alone

First we will just see if we can segment the nuclei

In [9]:

masks, flows, styles, diams = nuclei_model.eval(bs_nuc, channels=[0, 0],   #[0, 3] might work best
                                         diameter=15,
                                         cellprob_threshold=0,
                                         flow_threshold=.4)

In [10]:
viewer.layers.clear()
viewer.add_image(nuclei, colormap='magenta', blending='additive')
viewer.add_image(cyto, colormap='green', blending='additive')
viewer.add_image(bs_nuc, colormap='cyan', blending='additive', visible=False)
viewer.add_labels(masks)

<Labels layer 'masks' at 0x23d8936fee0>

## Segment cytoplasm alone

Use the cyto model to segment the cytoplasm with default diameter

In [11]:
cyto_masks, flows, styles, diams = cyto_model.eval(cyto, channels=[0, 0],
                                         diameter=30)
viewer.add_labels(cyto_masks)

<Labels layer 'cyto_masks' at 0x23d8a00c6d0>

Can you find anything that looks questionable?



## Segment cytoplasm, using both channels

The cellpose cyto models can use a second channel with nuclei to help find and serparate cells.

In the `channels` parameter, make the first index the channel to segment, the second index the channel with nuclei. It is important to start these at 1, not zero. 

In [12]:
data = np.stack([cyto, bs_nuc]) 
data.shape

(2, 384, 512)

In [13]:
data_masks, flows, styles, diams = cyto_model.eval(data, channels=[1, 2],  
                                         diameter=30)

viewer.add_labels(data_masks)

<Labels layer 'data_masks' at 0x23d89a807f0>

# Improving with flow_threshold

flow_threshold is (like cellprob_threshold) one of our tools for making the model more/less forgiving.  It defaults to 0.4, and ranges from 0 to 3.  Higher values allow objects that appear to be non-sensical according to the cellpose gradient images, but frequently it is useful for weird shapes that fail with the default settings.

In [14]:

data_masks, flows, styles, diams = cyto_model.eval(data, channels=[1, 2], 
                                         flow_threshold=1.0, diameter=30)

viewer.add_labels(data_masks)

<Labels layer 'data_masks [1]' at 0x23e21636560>